In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By # contains operators for the type of search we want to do
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

import re #for regular expressions

import pyktok as pyk #see https://github.com/dfreelon/pyktok for details
pyk.specify_browser('firefox') 

In [ ]:
class Tiktok_audit:
    def __init__(self, bot_user_num, bot_test_scenario, bot_email, bot_password):
        """
        bot_user_num        int; placeholder for future use
        bot_test_scenario   int; placeholder for future use
        bot_email           string; tiktok login email
        bot_password        string; tiktok login password
        """
        self.user_num = bot_user_num
        self.test_scenario = bot_test_scenario
        self.email = bot_email
        self.password = bot_password
        self.browser = webdriver.Firefox()
        
        self.batch_num = None  #1 batch = 30 posts
        self.videos_list = []
        
    def login(self):
        """
        Visits TikTok website and logs in to bot account
        """
        self.browser.get('https://www.tiktok.com/en')
        
        # click "use phone/number/email" on login page
        WebDriverWait(self.browser, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "css-1cp64nz-DivTextContainer")))
        self.browser.find_elements(By.CLASS_NAME, "css-1cp64nz-DivTextContainer")[1].click()

        # click "login with email or username"
        WebDriverWait(self.browser, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "css-1mgli76-ALink-StyledLink")))
        self.browser.find_element(By.CLASS_NAME, "css-1mgli76-ALink-StyledLink").click()

        # enter username and password then click log in
        WebDriverWait(self.browser, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "css-11to27l-InputContainer")))
        input_username = self.browser.find_element(By.CLASS_NAME, "css-11to27l-InputContainer")
        input_password = self.browser.find_element(By.CLASS_NAME, "css-wv3bkt-InputContainer")

        input_username.send_keys(self.email)
        input_password.send_keys(self.password)
        
        time.sleep(3)
                
        WebDriverWait(self.browser, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "css-11sviba-Button-StyledButton")))
        self.browser.find_element(By.CLASS_NAME, "css-11sviba-Button-StyledButton").click()
        
    def open_fyp(self):
        time.sleep(10)
        # Click on first video on FYP
        WebDriverWait(self.browser, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "css-11ma4ul-DivVideoPlayerContainer")))
        self.browser.find_element(By.CLASS_NAME, "css-11ma4ul-DivVideoPlayerContainer").click()
        
    def calc_vid_length(self):
        WebDriverWait(self.browser, 10).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[2]/div[4]/div/div[1]/div[3]/div[2]/div[2]")))
        lengthHTML = self.browser.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[4]/div/div[1]/div[3]/div[2]/div[2]").get_attribute("outerHTML")

        # Define a regular expression pattern to match the percentage (via ChatGPT)
        pattern = r'/(\d{2}:\d{2})'

        # Use re.search to find the pattern in the input string
        match = re.search(pattern, lengthHTML)

        # Check if the pattern is found and extract the time
        if match:
            time_value = match.group(1)
        else:
            print('No time found in the input string.')

        time_list = time_value.split(':')
        time_value = int(time_list[0])*60 + int(time_list[1]) #convert time to seconds

        return time_value
    
    def watch_batch(self):
        """
        Watches one batch of 30 TikTok videos and saves the metadata about each video
        """ 
        for i in range(0,31):
            # Next Video
            self.browser.find_element(By.CLASS_NAME, "css-1s9jpf8-ButtonBasicButtonContainer-StyledVideoSwitch").click()
            vid_length = self.calc_vid_length()
            if i % 2 == 0:
                #watch only 25% of the video
                time_to_watch = vid_length * 0.25
                time.sleep(time_to_watch)
            else:
                #watch full video
                time.sleep(vid_length)
            url = self.browser.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[4]/div/div[2]/div[1]/div/div[1]/div[2]/div/div[2]/p").text
            pyk.save_tiktok(f"{url}",
                            False,
                            'tiktok_videos.csv',
                            'firefox')

In [ ]:
test = Tiktok_audit(1,2,"selenium_315","cs315tiktok!")
#Note: complete the manual verification step at log in. Sometimes it glitches/doesn't let you; if so, re-run this.
test.login()
time.sleep(10)
test.open_fyp()
time.sleep(10)
test.watch_batch()